# Geo Data EDA & Feature Engineering

안녕하세요. 캐글 코리아에서 주최한 2회 대회를 즐겁게 하고 있는 사람으로서 제가 사용하고 있는 Feature Engineering을 공유하려고 합니다.

집값을 결정하는데 가장 중요한 요소는 입지라는 것은 모두 잘 아실거라고 생각합니다. 그래서 저는 기본 데이터에 대한 Feature Engineering 외에 위치에 관련된 Feature Engineering에 가장 시간을 쏟고 있는데요.

기본 데이터에 대한 EDA 및 Feature Engineering은 다른 분들이 공개한 좋은 커널이 많이 있으므로 생략하고, 위치에 관련된 zipcode, lat, long 데이터의 Feature Engineering에 집중해서 살펴보겠습니다.

보시고 도움이 됐다면 제 Kernel에 Vote 해주시면 감사하겠습니다! 

* [Baseline Model](#Baseline-Model)
* Geo Data Feature Engineering
  * [Zipcode Feature Engineering](#Zipcode-Feature-Engineering)
  * [PCA Transformation - Lat, Long](#PCA-Transformation---Lat,-Long)
  * [K-Means Clustering - Lat, Long](#K-Means-Clustering---Lat,-Long)
    * [Determine K by Elbow method](#Determine-K-by-Elbow-method)
    * [Determine K by CV Score](#Determine-K-by-CV-Score)
  * [Haversine Distance](#Haversine-Distance)
* [Conclusion](#Conclusion)

기본적인 코드는 아래의 커널들을 많이 참고했습니다.
* https://www.kaggle.com/chocozzz/house-price-prediction-eda-updated-2019-03-12
* https://www.kaggle.com/yeonmin/default-eda-stacking-introduction

위도(Lat), 경도(Long) 데이터를 기반으로한 Feature Engineering은 아래 글을 참고했습니다.

[Good Feature Building Techniques — Tricks for Kaggle — My Kaggle Code Repository](https://becominghuman.ai/good-feature-building-techniques-tricks-for-kaggle-my-kaggle-code-repository-c953b934f1e6)

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [1]:
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import xgboost as xgb
import lightgbm as lgb

# Set a few plotting defaults
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

pd.options.display.max_rows = 10000
pd.options.display.max_columns = 10000
pd.options.display.max_colwidth = 1000

In [1]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [1]:
def rmse_exp(y_true, y_pred):
    return np.sqrt(mean_squared_error(np.expm1(y_true), np.expm1(y_pred)))

def train_test_split(data, do_ohe=True):
    df = data.drop(['id','price','data'], axis=1).copy()
    cat_cols = df.select_dtypes('object').columns
    for col in cat_cols:
        if do_ohe:
            ohe_df = pd.get_dummies(df[[col]], prefix='ohe_'+col)
            df.drop(col, axis=1, inplace=True)
            df = pd.concat([df, ohe_df], axis=1)
        else:
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col])

    train_len = data[data['data'] == 'train'].shape[0]
    X_train = df.iloc[:train_len]
    X_test = df.iloc[train_len:]
    y_train = data[data['data'] == 'train']['price']
    
    return X_train, X_test, y_train

def get_oof_lgb(X_train, y_train, X_test, lgb_param, verbose_eval=False, return_cv_score_only=False):

    folds = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
    oof = np.zeros(len(X_train))
    predictions = np.zeros(len(X_test))
    feature_importance_df = pd.DataFrame()

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train.values, y_train.values)):
        if verbose_eval > 0: print(f'Fold : {fold_ + 1}')
        trn_data = lgb.Dataset(X_train.iloc[trn_idx], label=y_train.iloc[trn_idx])
        val_data = lgb.Dataset(X_train.iloc[val_idx], label=y_train.iloc[val_idx])

        num_round = 100000
        clf = lgb.train(lgb_param, trn_data, num_round, valid_sets=[trn_data, val_data],
                        verbose_eval=verbose_eval, early_stopping_rounds=200)
        oof[val_idx] = clf.predict(X_train.iloc[val_idx], num_iteration=clf.best_iteration)
        predictions += clf.predict(X_test, num_iteration=clf.best_iteration) / folds.n_splits
        
        cv_fold_score = rmse_exp(y_train.iloc[val_idx], oof[val_idx])
        
        if verbose_eval > 0: print(f'Fold {fold_ + 1} / CV-Score: {cv_fold_score:.6f}')
        
        fold_importance_df = pd.DataFrame()
        fold_importance_df['feature'] = X_train.columns.tolist()
        fold_importance_df['importance'] = clf.feature_importance('gain')
        fold_importance_df['fold'] = fold_ + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

    cv_score = rmse_exp(y_train, oof)
    print(f'CV-Score: {cv_score:.6f}')
    if return_cv_score_only: return cv_score
    else: return oof, predictions, cv_score, feature_importance_df
    
def plot_feature_importance(fi_df, num_feature=20):
    cols = (fi_df[['feature', 'importance']]
            .groupby('feature')
            .mean()
            .sort_values(by='importance', ascending=False)[:num_feature].index)
    best_features = fi_df.loc[fi_df.feature.isin(cols)]

    sns.barplot(x='importance', y='feature', data=best_features.sort_values(by='importance', ascending=False))
    plt.title('Feature Importances (averaged over folds)')
    plt.tight_layout()
    plt.show()
    
def plot_numeric_for_regression(df, field, target_field='price'):
    df = df[df[field].notnull()]

    fig = plt.figure(figsize = (16, 7))
    ax1 = plt.subplot(121)
    
    sns.distplot(df[df['data'] == 'train'][field], label='Train', hist_kws={'alpha': 0.5}, ax=ax1)
    sns.distplot(df[df['data'] == 'test'][field], label='Test', hist_kws={'alpha': 0.5}, ax=ax1)

    plt.xlabel(field)
    plt.ylabel('Density')
    plt.legend()
    
    ax2 = plt.subplot(122)
    
    df_copy = df[df['data'] == 'train'].copy()

    sns.scatterplot(x=field, y=target_field, data=df_copy, ax=ax2)
    
    plt.show()
    
def plot_categorical_for_regression(df, field, target_field='price', show_missing=True, missing_value='NA'):
    df_copy = df.copy()
    if show_missing: df_copy[field] = df_copy[field].fillna(missing_value)
    df_copy = df_copy[df_copy[field].notnull()]

    ax1_param = 121
    ax2_param = 122
    fig_size = (16, 7)
    if df_copy[field].nunique() > 30:
        ax1_param = 211
        ax2_param = 212
        fig_size = (16, 10)
    
    fig = plt.figure(figsize = fig_size)
    ax1 = plt.subplot(ax1_param)
    
    sns.countplot(x=field, hue='data', order=np.sort(df_copy[field].unique()), data=df_copy)
    plt.xticks(rotation=90, fontsize=11)
    
    ax2 = plt.subplot(ax2_param)
    
    df_copy = df_copy[df_copy['data'] == 'train']

    sns.boxplot(x=field, y=target_field, data=df_copy, order=np.sort(df_copy[field].unique()), ax=ax2)
    plt.xticks(rotation=90, fontsize=11)
    
    plt.show()
    
def load_original_data():
    train = pd.read_csv('../input/train.csv')
    test = pd.read_csv('../input/test.csv')

    train_copy = train.copy()
    train_copy['data'] = 'train'
    test_copy = test.copy()
    test_copy['data'] = 'test'
    test_copy['price'] = np.nan

    # remove outlier
    train_copy = train_copy[~((train_copy['sqft_living'] > 12000) & (train_copy['price'] < 3000000))].reset_index(drop=True)

    # concat train, test data to preprocess
    data = pd.concat([train_copy, test_copy], sort=False).reset_index(drop=True)
    data = data[train_copy.columns]

    data.drop('date', axis=1, inplace=True)
    data['zipcode'] = data['zipcode'].astype(str)

    # fix skew feature
    skew_columns = ['price']

    for c in skew_columns:
        data[c] = np.log1p(data[c])
        
    return data

In [1]:
data = load_original_data()

print(data.shape)
data.head()

# Baseline Model

LightGBM 5-Fold Out Of Fold Prediction을 Baseline model로 사용하겠습니다.

기본 데이터의 Baseline model CV 스코어는 아래와 같습니다. Categorical 데이터는 One Hot Encoding으로 처리하겠습니다.

In [1]:
X_train, X_test, y_train = train_test_split(data)
print(X_train.shape, X_test.shape)

lgb_param = {
    'objective': 'regression',
    'learning_rate': 0.05,
    'num_leaves': 15,
    'bagging_fraction': 0.7,
    'bagging_freq': 1,
    'feature_fraction': 0.7,
    'seed': RANDOM_SEED,
    'metric': ['rmse'],
}

oof, pred, cv_score, fi_df = get_oof_lgb(X_train, y_train, X_test, lgb_param)

In [1]:
plot_feature_importance(fi_df)

Feature Importance를 보면 zipcode의 98004, 98023, 98112, 98108이 중요한 feature로 나오는데요.

zipcode와 price의 boxplot을 그려보면 98004, 98112는 집값이 비싼 지역 98023, 98108은 집값이 낮은 지역임을 알 수 있습니다.

In [1]:
plot_categorical_for_regression(data, 'zipcode')

98004, 98112, 98023, 98108 지역이 어느 위치에 있는지 위도, 경도 데이터를 통해 확인해보겠습니다.

In [1]:
df = X_train
df['price'] = y_train

fig = plt.figure(figsize = (16, 12))

ax1 = plt.subplot(221)
sns.scatterplot(x='long', y='lat', hue='ohe_zipcode_98004', size='price', data=df, ax=ax1)

ax2 = plt.subplot(222)
sns.scatterplot(x='long', y='lat', hue='ohe_zipcode_98112', size='price', data=df, ax=ax2)

ax3 = plt.subplot(223)
sns.scatterplot(x='long', y='lat', hue='ohe_zipcode_98023', size='price', data=df, ax=ax3)

ax4 = plt.subplot(224)
sns.scatterplot(x='long', y='lat', hue='ohe_zipcode_98108', size='price', data=df, ax=ax4)

plt.show()

# Zipcode Feature Engineering

Zipcode는 다섯자리로 된 숫자형의 데이터인데, 데이터를 보면 앞의 두 자리는 98로 동일합니다.

그래서 뒤의 세 자리의 숫자를 여러가지 방법으로 쪼개서 새로운 feature를 만들어보겠습니다.  

In [1]:
data = load_original_data()

data['zipcode-3'] = 'z_' + data['zipcode'].str[2:3]
data['zipcode-4'] = 'z_' + data['zipcode'].str[3:4]
data['zipcode-5'] = 'z_' + data['zipcode'].str[4:5]
data['zipcode-34'] = 'z_' + data['zipcode'].str[2:4]
data['zipcode-45'] = 'z_' + data['zipcode'].str[3:5]
data['zipcode-35'] = 'z_' + data['zipcode'].str[2:3] + data['zipcode'].str[4:5]

print(data.shape)
data.head()

In [1]:
data['zipcode'] = 'z_' + data['zipcode']
sns.scatterplot(x='long', y='lat', hue='zipcode', hue_order=np.sort(data['zipcode'].unique()), data=data);

새롭게 만든 zipcode feature를 지도에서 살펴보겠습니다.

In [1]:
sns.scatterplot(x='long', y='lat', hue='zipcode-3', hue_order=np.sort(data['zipcode-3'].unique()), data=data);

In [1]:
sns.scatterplot(x='long', y='lat', hue='zipcode-4', hue_order=np.sort(data['zipcode-4'].unique()), data=data);

In [1]:
sns.scatterplot(x='long', y='lat', hue='zipcode-5', hue_order=np.sort(data['zipcode-5'].unique()), data=data);

In [1]:
sns.scatterplot(x='long', y='lat', hue='zipcode-34', hue_order=np.sort(data['zipcode-34'].unique()), data=data);

In [1]:
sns.scatterplot(x='long', y='lat', hue='zipcode-45', hue_order=np.sort(data['zipcode-45'].unique()), data=data);

In [1]:
sns.scatterplot(x='long', y='lat', hue='zipcode-35', hue_order=np.sort(data['zipcode-35'].unique()), data=data);

In [1]:
X_train, X_test, y_train = train_test_split(data)
print(X_train.shape, X_test.shape)

lgb_param = {
    'objective': 'regression',
    'learning_rate': 0.05,
    'num_leaves': 15,
    'bagging_fraction': 0.7,
    'bagging_freq': 1,
    'feature_fraction': 0.7,
    'seed': RANDOM_SEED,
    'metric': ['rmse'],
}

oof, pred, cv_score, fi_df = get_oof_lgb(X_train, y_train, X_test, lgb_param)

새롭게 만든 zipcode feature로 CV Score가 더 좋아진 것을 확인할 수 있습니다.

In [1]:
plot_feature_importance(fi_df)

Feature Importance를 보면 zipcode-35의 18, zipcode-5의 8이 중요한 feature로 나오고 boxplot을 그려보면 집값이 싼 지역임을 알 수 있습니다.

In [1]:
plot_categorical_for_regression(data, 'zipcode-35')

In [1]:
plot_categorical_for_regression(data, 'zipcode-5')

# PCA Transformation - Lat, Long

PCA는 차원축소에 주로 사용되는 알고리즘입니다. 위도, 경도의 데이터만 보면 2차원의 데이터인데, 이 데이터를 차원축소는 하지 않고 2차원 그대로 PCA Transformation을 하면 원본 데이터를 변형해서 새로운 feature를 만들 수 있습니다.

In [1]:
# pca for lat, long
data = load_original_data()

coord = data[['lat','long']]
pca = PCA(n_components=2)
pca.fit(coord)

coord_pca = pca.transform(coord)

data['coord_pca1'] = coord_pca[:, 0]
data['coord_pca2'] = coord_pca[:, 1]

이렇게 만들어진 feature를 2차원에 그려보면 원본 데이터가 변형되서 새로운 feature로 만들어진 것을 확인할 수 있습니다.

In [1]:
sns.scatterplot(x='coord_pca2', y='coord_pca1', hue='price', data=data);

In [1]:
X_train, X_test, y_train = train_test_split(data)
print(X_train.shape, X_test.shape)

lgb_param = {
    'objective': 'regression',
    'learning_rate': 0.05,
    'num_leaves': 15,
    'bagging_fraction': 0.7,
    'bagging_freq': 1,
    'feature_fraction': 0.7,
    'seed': RANDOM_SEED,
    'metric': ['rmse'],
}

oof, pred, cv_score, fi_df = get_oof_lgb(X_train, y_train, X_test, lgb_param)

PCA를 통해 만든 feature로 CV Score가 더 좋아진 것을 확인할 수 있고, 아래 feature importance에서도 중요한 feature로 나오는 것을 알 수 있습니다.

In [1]:
plot_feature_importance(fi_df)

In [1]:
plot_numeric_for_regression(data, 'coord_pca2')
plot_numeric_for_regression(data, 'coord_pca1')

# K-Means Clustering - Lat, Long

K-Means Clustering은 Clustering에 사용되는 비지도학습 알고리즘 중의 하나입니다. 위도, 경도 데이터를 K-Means Clustering 하면 가까운 지역끼리 Cluster가 만들어지기 때문에 zipcode와 유사한 개념의 새로운 feature를 만들 수 있습니다.

# Determine K by Elbow method

K-Means Clustering에서 중요한 건 K 값을 어떻게 결정하느냐 인데요. 일반적으로 많이 쓰이는 방법은 K 값을 늘려가면서 여러 개 돌려보면, Cluster 간의 거리의 합을 나타내는 inertia가 급격히 떨어지는 구간이 생기는데 이 지점의 K 값을 많이 사용합니다. 이런 방식을 Elbow Method라고 합니다. 먼저 Elbow Method를 통해 K 값을 결정해보겠습니다.

In [1]:
inertia_arr = []

k_range = range(2, 16)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=RANDOM_SEED).fit(coord)
 
    # Sum of distances of samples to their closest cluster center
    interia = kmeans.inertia_
    print ("k:",k, " cost:", interia)
    inertia_arr.append(interia)
    
inertia_arr = np.array(inertia_arr)

plt.plot(k_range, inertia_arr)
plt.vlines(5, ymin=inertia_arr.min()*0.9999, ymax=inertia_arr.max()*1.0003, linestyles='--', colors='b')
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia');

위의 그래프를 보면 5가 가장 적절한 K 값으로 볼 수 있습니다.

In [1]:
data = load_original_data()

# kmeans for lat, long
kmeans = KMeans(n_clusters=5, random_state=RANDOM_SEED).fit(coord)
coord_cluster = kmeans.predict(coord)
data['coord_cluster'] = coord_cluster
data['coord_cluster'] = data['coord_cluster'].map(lambda x: 'c_' + str(x).rjust(2, '0'))

X_train, X_test, y_train = train_test_split(data)
print(X_train.shape, X_test.shape)

lgb_param = {
    'objective': 'regression',
    'learning_rate': 0.05,
    'num_leaves': 15,
    'bagging_fraction': 0.7,
    'bagging_freq': 1,
    'feature_fraction': 0.7,
    'seed': RANDOM_SEED,
    'metric': ['rmse'],
}

oof, pred, cv_score, fi_df = get_oof_lgb(X_train, y_train, X_test, lgb_param)

# Determine K by CV Score

또, 다른 방식으로 K 값을 결정해볼 수 도 있는데요. K-Means Clustering으로 만든 feature를 Regression Model에서 결국 사용하기 때문에 K 값도 하나의 하이퍼 파라미터로 보고 CV Score가 가장 낮게 나오는 K를 선택해서 사용할 수도 있습니다.

In [1]:
k_range = range(2, 80, 5)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=RANDOM_SEED).fit(coord)
    coord_cluster = kmeans.predict(coord)
    data['coord_cluster'] = coord_cluster
    data['coord_cluster'] = data['coord_cluster'].map(lambda x: str(x).rjust(2, '0'))
    
    X_train, X_test, y_train = train_test_split(data)

    lgb_param = {
        'objective': 'regression',
        'learning_rate': 0.05,
        'num_leaves': 15,
        'bagging_fraction': 0.7,
        'bagging_freq': 1,
        'feature_fraction': 0.7,
        'seed': RANDOM_SEED,
        'metric': ['rmse'],
    }

    print('K :', k)
    get_oof_lgb(X_train, y_train, X_test, lgb_param)
    print()

In [1]:
k_range = range(28, 37)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=RANDOM_SEED).fit(coord)
    coord_cluster = kmeans.predict(coord)
    data['coord_cluster'] = coord_cluster
    data['coord_cluster'] = data['coord_cluster'].map(lambda x: str(x).rjust(2, '0'))
    
    X_train, X_test, y_train = train_test_split(data)

    lgb_param = {
        'objective': 'regression',
        'learning_rate': 0.05,
        'num_leaves': 15,
        'bagging_fraction': 0.7,
        'bagging_freq': 1,
        'feature_fraction': 0.7,
        'seed': RANDOM_SEED,
        'metric': ['rmse'],
    }

    print('K :', k)
    get_oof_lgb(X_train, y_train, X_test, lgb_param)
    print()

K 값이 32일 때 CV Score가 가장 좋게 나오는 것을 알 수 있습니다. Elbow Method로 K 값을 선택했을 때보다 CV Score 기준으로 K를 선택할 때가 더 모델 성능이 좋기 때문에 최종 K 값은 32로 사용하겠습니다.  

In [1]:
# kmeans for lat, long
kmeans = KMeans(n_clusters=32, random_state=RANDOM_SEED).fit(coord)
coord_cluster = kmeans.predict(coord)
data['coord_cluster'] = coord_cluster
data['coord_cluster'] = data['coord_cluster'].map(lambda x: 'c_' + str(x).rjust(2, '0'))

In [1]:
sns.scatterplot(x='long', y='lat', hue='coord_cluster', hue_order=np.sort(data['coord_cluster'].unique()), data=data);

In [1]:
X_train, X_test, y_train = train_test_split(data)
print(X_train.shape, X_test.shape)

lgb_param = {
    'objective': 'regression',
    'learning_rate': 0.05,
    'num_leaves': 15,
    'bagging_fraction': 0.7,
    'bagging_freq': 1,
    'feature_fraction': 0.7,
    'seed': RANDOM_SEED,
    'metric': ['rmse'],
}

oof, pred, cv_score, fi_df = get_oof_lgb(X_train, y_train, X_test, lgb_param)

In [1]:
plot_feature_importance(fi_df)

Feature Importance를 보면 11번 cluster가 중요한 feature임을 알 수 있습니다. 아래 그래프에서 11번 cluster의 위치를 확인해볼 수 있습니다.

In [1]:
df = X_train
df['price'] = y_train
sns.scatterplot(x='long', y='lat', hue='ohe_coord_cluster_c_11', data=df);

Cluster의 boxplot을 확인해보면 11번 cluster가 집값이 가장 비싼 지역임을 알 수 있습니다.

In [1]:
plot_categorical_for_regression(data, 'coord_cluster')

# [Haversine Distance](https://en.wikipedia.org/wiki/Haversine_formula)

마지막으로 Haversine Distance를 사용한 feature 생성입니다. Haversine Distance는 두 개의 위도, 경도 좌표에서 지구의 곡률을 고려해 두 좌표 간의 거리를 구하는 방법입니다.

아래의 function으로 Haversine Distance를 구할 수 있습니다.

In [1]:
def haversine_array(lat1, lng1, lat2, lng2): 
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2)) 
    AVG_EARTH_RADIUS = 6371 # in km 
    lat = lat2 - lat1 
    lng = lng2 - lng1 
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2 
    h = 2 * AVG_EARTH_RADIUS * np.arcsin(np.sqrt(d)) 
    return h

주어진 데이터에서 위도, 경도의 최소값과, 최대값 두 개의 좌표에 대해 거리를 구해보면 113.88km가 나오는데, 가장 먼 거리의 거리가 이 정도라는 얘기가 되겠네요.

In [1]:
print(data['lat'].min(), data['lat'].max(), data['long'].min(), data['long'].max())

haversine_dist = haversine_array(data['lat'].min(), data['long'].min(), data['lat'].max(), data['long'].max())
print(f'max distance: {haversine_dist:.2f}km')

아래는 id가 0인 집과 전체 집과의 거리를 구한 데이터입니다. 0번 집과 가까운 이웃은 집값이 비슷할 확률이 높기 때문에 거리를 기반으로 가까운 이웃집의 데이터를 찾을 수 있습니다. 이런 식으로 전체 집들의 Haversine Distance를 구하면 나와 가까운 이웃집들의 정보를 새로운 feature로 활용할 수 있습니다.

In [1]:
neighbor_df = pd.DataFrame()
lat2 = data['lat'].values
long2 = data['long'].values

lat1 = data.loc[0, 'lat'] # id = 0 house lat
long1 = data.loc[0, 'long'] # id = 0 house long
dist_arr = haversine_array(lat1, long1, lat2, long2)
neighbor_df = pd.DataFrame({
    'id': np.tile(np.array([data.loc[0, 'id']]), data.shape[0]),
    'neighbor_id': data['id'],
    'neighbor_lat': lat2,
    'neighbor_long': long2,
    'distance': dist_arr,
})
    
print(neighbor_df.shape)
neighbor_df.head()

아래는 0번 집의 반경 5km 이내의 이웃들을 그려본 그래프입니다. 

In [1]:
neighbor_df['neighbor_10km'] = neighbor_df['distance'] <= 5
sns.scatterplot(x='neighbor_long', y='neighbor_lat', hue='neighbor_10km', data=neighbor_df);

# Conclusion

지금까지 위치 데이터를 활용한 Feature Engineering을 살펴봤습니다. 특히, 마지막의 Haversine Distance를 활용하면 각 집에서 가까운 이웃집을 계산할 수 있고, 이를 바탕으로 다양한 feature를 만들어 낼 수 있습니다.

저같은 경우 이러한 Geo Data Feature Engineering과 Stacking Ensemble을 통해 퍼블릭 리더보드 기준으로 RMSE 스코어가 96000대까지 성능이 나오는 모델을 만들 수 있었습니다. 제가 공유한 내용이 다른 분들에게도 도움이 됐으면 좋겠네요.

감사합니다.